In [6]:
'''
Created by Omar Padierna "Para11ax" on Jan 1 2019

 This program is free software: you can redistribute it and/or modify
 it under the terms of the GNU General Public License as published by
 the Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 This program is distributed in the hope that it will be useful,
 but WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 GNU General Public License for more details.

'''


import cv2
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image

#============================================
# Camera calibration
#============================================

#Define size of chessboard target. 

chessboard_size = (6,9)

#Define arrays to save detected points
obj_points = [] #3D points in real world space 
img_points = [] #3D points in image plane

#Prepare grid and points to display

objp = np.zeros((np.prod(chessboard_size),3),dtype=np.float32)


objp[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1,2)

#read images

calibration_paths = glob.glob('./img3/*')

#Iterate over images to find intrinsic matrix
for image_path in tqdm(calibration_paths):

	#Load image
	image = cv2.imread(image_path)
	gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	print("Image loaded, Analizying...")
	#find chessboard corners
	ret,corners = cv2.findChessboardCorners(gray_image, chessboard_size, None)

	if ret == True:
		print("Chessboard detected!")
		print(image_path)
		#define criteria for subpixel accuracy
		criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
		#refine corner location (to subpixel accuracy) based on criteria.
		cv2.cornerSubPix(gray_image, corners, (5,5), (-1,-1), criteria)
		obj_points.append(objp)
		img_points.append(corners)

#Calibrate camera
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points,gray_image.shape[::-1], None, None)

#Save parameters into numpy file
np.save("./camera_params/ret", ret)
np.save("./camera_params/K", K)
np.save("./camera_params/dist", dist)
np.save("./camera_params/rvecs", rvecs)
np.save("./camera_params/tvecs", tvecs)

#Get exif data in order to get focal length. 
#exif_img = PIL.Image.open(calibration_paths[0])

#exif_data = {
#	PIL.ExifTags.TAGS[k]:v
#	for k, v in exif_img._getexif().items()
#	if k in PIL.ExifTags.TAGS}

#Get focal length in tuple form
#focal_length_exif = exif_data['FocalLength']

#Get focal length in decimal form
focal_length = 4449/1296

#Save focal length
np.save("./camera_params/FocalLength", focal_length)

#Calculate projection error. 
mean_error = 0
for i in range(len(obj_points)):
	img_points2, _ = cv2.projectPoints(obj_points[i],rvecs[i],tvecs[i], K, dist)
	error = cv2.norm(img_points[i], img_points2, cv2.NORM_L2)/len(img_points2)
	mean_error += error

total_error = mean_error/len(obj_points)
print (total_error)


 11%|█████████▏                                                                         | 2/18 [00:00<00:01, 10.48it/s]

Image loaded, Analizying...
Chessboard detected!
./img3\left01.jpg
Image loaded, Analizying...
Chessboard detected!
./img3\left02.jpg
Image loaded, Analizying...


 22%|██████████████████▍                                                                | 4/18 [00:00<00:01,  9.87it/s]

Chessboard detected!
./img3\left03.jpg
Image loaded, Analizying...
Chessboard detected!
./img3\left04.jpg
Image loaded, Analizying...


 39%|████████████████████████████████▎                                                  | 7/18 [00:00<00:01,  9.73it/s]

...

 89%|████████████████████████████████████████████████████████████████████████▉         | 16/18 [00:01<00:00,  8.89it/s]

Chessboard detected!
./img3\right06.jpg
Image loaded, Analizying...
Chessboard detected!
./img3\right07.jpg
Image loaded, Analizying...


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:02<00:00,  8.89it/s]

Chessboard detected!
./img3\right08.jpg
Image loaded, Analizying...
Chessboard detected!
./img3\right09.jpg


0.026347073279672766
